In [1]:
import subprocess
import sys

# Check GPU availability
subprocess.run(['nvidia-smi'], check=False)

print("\n" + "="*80)
print("🔍 ENVIRONMENT CHECK")
print("="*80)

Thu Oct 16 20:02:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# N8N Workflow Generator - Training Pipeline

This notebook will:
1. Install required dependencies
2. Build a training dataset from your n8n workflows
3. Fine-tune a Llama model using QLoRA
4. Save the trained model for inference

## Step 1: Install Dependencies

In [2]:
import warnings
warnings.filterwarnings('ignore')

print("🔧 Applying HuggingFace Hub patches...\n")

# Patch 1: Suppress 404 errors for additional_chat_templates
try:
    from transformers.utils import hub as transformers_hub
    original_list_repo_templates = transformers_hub.list_repo_templates
    
    def patched_list_repo_templates(repo_id, local_files_only=False, revision=None, cache_dir=None):
        try:
            return original_list_repo_templates(repo_id, local_files_only, revision, cache_dir)
        except Exception as e:
            if "additional_chat_templates" in str(e) or "404" in str(e):
                return []
            raise
    
    transformers_hub.list_repo_templates = patched_list_repo_templates
    print("✅ Patched transformers.utils.hub.list_repo_templates")
except Exception as e:
    print(f"⚠️ Could not patch list_repo_templates: {e}")

# Patch 2: Suppress 404 errors in huggingface_hub
try:
    import huggingface_hub.utils._http as hf_http
    original_hf_raise = hf_http.hf_raise_for_status
    
    def patched_hf_raise(response, endpoint_name=None):
        if hasattr(response, 'status_code') and response.status_code == 404:
            if hasattr(response, 'url') and "additional_chat_templates" in str(response.url):
                return
        return original_hf_raise(response, endpoint_name)
    
    hf_http.hf_raise_for_status = patched_hf_raise
    print("✅ Patched huggingface_hub.utils._http.hf_raise_for_status")
except Exception as e:
    print(f"⚠️ Could not patch hf_raise_for_status: {e}")

print("\n✅ All HuggingFace patches applied!")

🔧 Applying HuggingFace Hub patches...

✅ Patched transformers.utils.hub.list_repo_templates
✅ Patched huggingface_hub.utils._http.hf_raise_for_status

✅ All HuggingFace patches applied!
✅ Patched transformers.utils.hub.list_repo_templates
✅ Patched huggingface_hub.utils._http.hf_raise_for_status

✅ All HuggingFace patches applied!


In [3]:
# RESTART THE KERNEL FIRST (in Kaggle: Runtime > Restart Runtime)
# Then run this cell

print("🔄 POST-RESTART: Installing minimal required packages...\n")

# Only install what's NOT already in Kaggle
import subprocess
import sys

# Kaggle comes with torch, transformers, accelerate, datasets pre-installed
# We only need to add bitsandbytes and peft
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "bitsandbytes", "peft"], check=False)

print("\n✅ Installation complete! Proceed to next cell.")

🔄 POST-RESTART: Installing minimal required packages...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.



✅ Installation complete! Proceed to next cell.


## Step 2: Upload Your Workflow Files

Upload your `workflows/` folder to Kaggle (or the files are already synced from your GitHub repo)

In [5]:
# Check if workflows directory exists
import os
if os.path.exists('/kaggle/working/workflows'):
    print("✓ Workflows folder found!")
    workflow_count = len([f for f in os.listdir('/kaggle/working/workflows') if f.endswith('.json')])
    print(f"Found {workflow_count} workflow files")
else:
    print("⚠ Workflows folder not found. Please upload your workflows/ folder to Kaggle.")

⚠ Workflows folder not found. Please upload your workflows/ folder to Kaggle.


In [12]:
# Upload dataset manually or check Kaggle input directory
import os

# Check what's available in /kaggle/input/
print("Checking /kaggle/input/ directory...")
if os.path.exists('/kaggle/input'):
    for item in os.listdir('/kaggle/input'):
        full_path = os.path.join('/kaggle/input', item)
        if os.path.isdir(full_path):
            print(f"\nDataset: {item}")
            print(f"  Files: {os.listdir(full_path)}")
else:
    print("⚠ /kaggle/input does not exist - you may be running locally")
    print(f"Current directory: {os.getcwd()}")
    
print("\n💡 SOLUTION:")
print("Since you're in Kaggle environment, you need to:")
print("1. Upload dataset.jsonl as a Kaggle dataset")
print("2. Add it to this notebook via 'Add Data'")
print("3. OR: Copy dataset.jsonl to /kaggle/working/ manually")

Checking /kaggle/input/ directory...

Dataset: uploaded-data
  Files: ['dataset.jsonl', 'qlora_train.py']

💡 SOLUTION:
Since you're in Kaggle environment, you need to:
1. Upload dataset.jsonl as a Kaggle dataset
2. Add it to this notebook via 'Add Data'
3. OR: Copy dataset.jsonl to /kaggle/working/ manually


## Step 3: Build Training Dataset

In [6]:
import json
import os
from pathlib import Path

# Dataset builder code
def extract_workflow_info(workflow_data):
    """Extract key information from workflow for prompt generation."""
    nodes = workflow_data.get('nodes', [])
    
    node_types = [node.get('type', 'Unknown') for node in nodes]
    node_names = [node.get('name', '') for node in nodes]
    
    # Try to infer purpose from node types and names
    actions = []
    for node in nodes:
        node_type = node.get('type', '')
        node_name = node.get('name', '')
        if node_type and node_type not in ['n8n-nodes-base.start', 'n8n-nodes-base.manualTrigger']:
            actions.append(f"{node_type} ({node_name})")
    
    return {
        'node_types': node_types,
        'node_names': node_names,
        'actions': actions,
        'node_count': len(nodes)
    }

def generate_prompt_from_workflow(workflow_data, workflow_name):
    """Generate a natural language prompt from workflow structure."""
    info = extract_workflow_info(workflow_data)
    
    # Create a descriptive prompt based on the workflow
    if info['node_count'] <= 2:
        return f"Create a simple workflow with {', '.join(info['node_types'])}"
    
    actions_str = ' and '.join(info['actions'][:3])  # First 3 actions
    return f"Create a workflow that uses {actions_str}"

def build_dataset(workflows_dir, output_file):
    """Build training dataset from workflow JSON files."""
    dataset = []
    
    workflows_path = Path(workflows_dir)
    if not workflows_path.exists():
        print(f"Error: {workflows_dir} not found!")
        return []
    
    workflow_files = list(workflows_path.glob('*.json'))
    print(f"Found {len(workflow_files)} workflow files")
    
    for workflow_file in workflow_files:
        try:
            with open(workflow_file, 'r', encoding='utf-8') as f:
                workflow_data = json.load(f)
            
            prompt = generate_prompt_from_workflow(workflow_data, workflow_file.stem)
            workflow_json = json.dumps(workflow_data, indent=2)
            
            dataset.append({
                'prompt': prompt,
                'workflow': workflow_json
            })
            
        except Exception as e:
            print(f"Error processing {workflow_file.name}: {e}")
            continue
    
    # Save dataset
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, indent=2)
    
    print(f"\n✓ Built dataset with {len(dataset)} examples")
    print(f"✓ Saved to {output_file}")
    
    return dataset

# Build the dataset
workflows_dir = '/kaggle/working/workflows'
output_file = '/kaggle/working/training_dataset.json'

dataset = build_dataset(workflows_dir, output_file)

# Show sample
if dataset:
    print("\n--- Sample Training Example ---")
    print(f"Prompt: {dataset[0]['prompt']}")
    print(f"Workflow: {dataset[0]['workflow'][:200]}...")  # First 200 chars

Error: /kaggle/working/workflows not found!


## Step 4: Prepare Training Data for Model

In [4]:
import json
from datasets import Dataset

# Load the dataset from Kaggle input
formatted_data = []

# Use the correct Kaggle input path
dataset_path = '/kaggle/input/testingdata/dataset.jsonl'

print(f"✓ Loading dataset from: {dataset_path}")

# Read JSONL file
with open(dataset_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # Skip empty lines
            item = json.loads(line.strip())
            # Convert workflow dict to JSON string if it's not already
            workflow_str = json.dumps(item['workflow']) if isinstance(item['workflow'], dict) else item['workflow']
            
            formatted_data.append({
                'text': f"""<|system|>
You are an n8n workflow generator. Convert natural language descriptions into valid n8n workflow JSON.
<|user|>
{item['prompt']}
<|assistant|>
{workflow_str}"""
            })

# Create HuggingFace dataset
train_dataset = Dataset.from_list(formatted_data)

print(f"✓ Created dataset with {len(train_dataset)} examples")
print(f"✓ Sample length: {len(formatted_data[0]['text'])} characters")
print(f"✓ Ready for training!")

✓ Loading dataset from: /kaggle/input/testingdata/dataset.jsonl
✓ Created dataset with 1000 examples
✓ Sample length: 11231 characters
✓ Ready for training!
✓ Created dataset with 1000 examples
✓ Sample length: 11231 characters
✓ Ready for training!


## Step 5: Load Base Model with QLoRA Configuration

In [10]:
print("🚀 Loading Model and Tokenizer...\n")

import torch
import os
from pathlib import Path

# Download tokenizer files
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer_cache = Path("/tmp/tinyllama_tokenizer")
tokenizer_cache.mkdir(parents=True, exist_ok=True)

print("📥 Downloading tokenizer files...")
from huggingface_hub import hf_hub_download
import shutil

for fname in ["tokenizer.model", "tokenizer.json", "tokenizer_config.json", "special_tokens_map.json", "config.json"]:
    try:
        fpath = hf_hub_download(repo_id=model_name, filename=fname, cache_dir=str(tokenizer_cache), local_files_only=False)
        shutil.copy(fpath, tokenizer_cache / fname)
    except:
        pass

# Load tokenizer
print("📥 Loading tokenizer...")
from transformers import LlamaTokenizer

tokenizer_file = tokenizer_cache / "tokenizer.model"
tokenizer = LlamaTokenizer(vocab_file=str(tokenizer_file))
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("✅ Tokenizer loaded")

# Load model in fp16 (no quantization for now - bitsandbytes not working)
print("\n📥 Loading model in fp16 (this may take 1-2 minutes)...")
from transformers import AutoModelForCausalLM

print("   ⚠️  Using fp16 without 4-bit quantization (will use ~2-3GB VRAM per GPU)")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    load_in_8bit=False,
    low_cpu_mem_usage=True,
)

# Prepare for training
model.config.use_cache = False
model.gradient_checkpointing_enable()

print("\n✅ Model loaded successfully!")
print(f"   Device: {next(model.parameters()).device}")
print(f"   Tokenizer vocab size: {len(tokenizer)}")
print(f"   Model dtype: {next(model.parameters()).dtype}")

🚀 Loading Model and Tokenizer...

📥 Downloading tokenizer files...
📥 Loading tokenizer...
✅ Tokenizer loaded

📥 Loading model in fp16 (this may take 1-2 minutes)...
   ⚠️  Using fp16 without 4-bit quantization (will use ~2-3GB VRAM per GPU)
📥 Loading tokenizer...
✅ Tokenizer loaded

📥 Loading model in fp16 (this may take 1-2 minutes)...
   ⚠️  Using fp16 without 4-bit quantization (will use ~2-3GB VRAM per GPU)


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


✅ Model loaded successfully!
   Device: cuda:0
   Tokenizer vocab size: 32000
   Model dtype: torch.float16


## Step 6: Configure LoRA Adapters

In [11]:
from peft import LoraConfig, get_peft_model

print("🎯 Configuring LoRA adapters...\n")

# LoRA config optimized for small model (TinyLlama)
lora_config = LoraConfig(
    r=8,                                    # Reduced rank for 1.1B model
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],   # Fewer modules = faster training
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

try:
    model = get_peft_model(model, lora_config)
    print("✅ LoRA adapters applied!")
    print("\n📊 Trainable parameters:")
    model.print_trainable_parameters()
except Exception as e:
    print(f"❌ Failed to apply LoRA: {e}")
    raise

print("\n✅ Model ready for training!")

🎯 Configuring LoRA adapters...

✅ LoRA adapters applied!

📊 Trainable parameters:
trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023

✅ Model ready for training!
✅ LoRA adapters applied!

📊 Trainable parameters:
trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023

✅ Model ready for training!


## Step 7: Configure Training Parameters

In [12]:
from transformers import TrainingArguments

print("⚙️  Configuring training parameters...\n")

# Memory-optimized training config
training_args = TrainingArguments(
    output_dir="/kaggle/working/checkpoints",
    num_train_epochs=2,                    # Reduced from 3 to 2 for faster iteration
    per_device_train_batch_size=1,         # Minimal batch size
    gradient_accumulation_steps=4,         # Effective batch size = 4
    learning_rate=1e-4,                    # Lower LR for stability
    warmup_steps=50,
    max_steps=-1,                          # Use num_epochs instead
    fp16=True,                             # Mixed precision
    save_strategy="steps",
    save_steps=10,                         # Save checkpoint every 10 steps
    save_total_limit=3,                    # Keep only 3 latest checkpoints
    logging_steps=5,
    logging_first_step=True,
    disable_tqdm=False,
    gradient_checkpointing=True,           # Save VRAM
    optim="paged_adamw_8bit",              # 8-bit optimizer
    max_grad_norm=1.0,
    lr_scheduler_type="linear",
    report_to="none",                      # No wandb
    remove_unused_columns=False,
    dataloader_pin_memory=True,
    seed=42,
)

print("✅ Training configuration:")
print(f"   • Batch size: 1 (per device)")
print(f"   • Gradient accumulation: 4")
print(f"   • Effective batch size: 4")
print(f"   • Learning rate: {training_args.learning_rate}")
print(f"   • Save checkpoint every: {training_args.save_steps} steps")
print(f"   • Keep {training_args.save_total_limit} latest checkpoints")
print(f"   • Mixed precision (fp16): Enabled")
print(f"   • Gradient checkpointing: Enabled")
print(f"   • Checkpoint directory: {training_args.output_dir}")

⚙️  Configuring training parameters...

✅ Training configuration:
   • Batch size: 1 (per device)
   • Gradient accumulation: 4
   • Effective batch size: 4
   • Learning rate: 0.0001
   • Save checkpoint every: 10 steps
   • Keep 3 latest checkpoints
   • Mixed precision (fp16): Enabled
   • Gradient checkpointing: Enabled
   • Checkpoint directory: /kaggle/working/checkpoints


## Step 8: Start Training! 🚀

This will take approximately **45-90 minutes** depending on GPU speed.


### ⏱️ Time Expectation:
- **First 100 steps**: ~20-30 minutes (this is normal!)
- **Full training**: 45-90 minutes total
- **Updates every**: 5 steps (you'll see ~75 progress updates)

In [13]:
from transformers import Trainer, DataCollatorForLanguageModeling
import os
import time
from datetime import datetime

print("🚀 STARTING TRAINING PIPELINE\n")
print("="*80)

# ==================== TOKENIZATION ====================
print("📝 Tokenizing dataset...")

def tokenize_function(examples):
    result = tokenizer(
        examples["text"],
        padding="max_length",
        max_length=512,
        truncation=True,
        return_tensors=None,
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=100,
    remove_columns=["text"],
    desc="Tokenizing dataset",
)

print(f"✅ Tokenized {len(tokenized_dataset)} examples")
print(f"   Sample length: {len(tokenized_dataset[0]['input_ids'])} tokens\n")

# ==================== DATA COLLATOR ====================
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    return_tensors="pt",
)

# ==================== CHECKPOINT DETECTION ====================
checkpoint_dir = "/kaggle/working/checkpoints"
resume_from_checkpoint = None

if os.path.exists(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")]
    if checkpoints:
        latest = sorted(checkpoints, key=lambda x: int(x.split("-")[1]))[-1]
        resume_from_checkpoint = os.path.join(checkpoint_dir, latest)
        step = latest.split("-")[1]
        print(f"🔄 RESUMING FROM CHECKPOINT")
        print(f"   Checkpoint: {latest}")
        print(f"   Step: {step}\n")

# ==================== INITIALIZE TRAINER ====================
print("⚙️  Initializing Trainer...")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("✅ Trainer initialized\n")

# ==================== START TRAINING ====================
print("="*80)
print("📊 TRAINING START TIME:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("="*80 + "\n")

start_time = time.time()

try:
    trainer.train(resume_from_checkpoint=resume_from_checkpoint)
    
    elapsed = (time.time() - start_time) / 60
    print("\n" + "="*80)
    print("✅ TRAINING COMPLETED SUCCESSFULLY!")
    print(f"   Duration: {elapsed:.1f} minutes")
    print(f"   End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*80)
    
except KeyboardInterrupt:
    elapsed = (time.time() - start_time) / 60
    print("\n⚠️ Training interrupted by user")
    print(f"   Elapsed: {elapsed:.1f} minutes")
    print("   ✅ Checkpoints saved - run again to resume!")
    
except Exception as e:
    elapsed = (time.time() - start_time) / 60
    print(f"\n❌ Training failed after {elapsed:.1f} minutes")
    print(f"   Error: {e}")
    print("   ✅ Checkpoints saved - run again to resume!")
    raise

🚀 STARTING TRAINING PIPELINE

📝 Tokenizing dataset...


Tokenizing dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


✅ Tokenized 1000 examples
   Sample length: 512 tokens

⚙️  Initializing Trainer...
✅ Trainer initialized

📊 TRAINING START TIME: 2025-10-16 20:07:02



Step,Training Loss
1,2.047000
5,1.841800
10,1.842200
15,2.013000
20,2.065500
25,1.992300
30,1.915800
35,1.710700
40,1.872900
45,1.787200



✅ TRAINING COMPLETED SUCCESSFULLY!
   Duration: 8.2 minutes
   End time: 2025-10-16 20:15:13


In [7]:
# Check if any checkpoints were saved before the interruption
import os

checkpoint_dir = "/kaggle/working/n8n-workflow-generator"

if os.path.exists(checkpoint_dir):
    contents = os.listdir(checkpoint_dir)
    checkpoints = [d for d in contents if d.startswith("checkpoint-")]
    
    if checkpoints:
        print(f"✅ Found {len(checkpoints)} checkpoint(s):")
        for cp in sorted(checkpoints):
            print(f"   - {cp}")
        print("\n💡 You can resume training from the last checkpoint!")
    else:
        print("❌ No checkpoints saved yet")
        print("💡 The interruption happened before the first checkpoint (step 25)")
else:
    print("❌ Checkpoint directory doesn't exist yet")
    print("💡 Training was interrupted before any checkpoints could be saved")

❌ Checkpoint directory doesn't exist yet
💡 Training was interrupted before any checkpoints could be saved


## Step 9: Save the Trained Model

In [14]:
# Save the fine-tuned model
output_dir = "/kaggle/working/n8n-workflow-generator-final"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ Model saved to {output_dir}")
print("\n📦 To download the model:")
print("1. Go to Kaggle's Output tab")
print("2. Download the 'n8n-workflow-generator-final' folder")
print("3. Place it in your project's 'models/' directory")

✓ Model saved to /kaggle/working/n8n-workflow-generator-final

📦 To download the model:
1. Go to Kaggle's Output tab
2. Download the 'n8n-workflow-generator-final' folder
3. Place it in your project's 'models/' directory


## Step 10: Test the Model (Optional)

Let's test the model with a sample prompt to see if it generates valid n8n workflows!

In [15]:
# Test the model
test_prompt = "Create a workflow that sends an email notification"

inputs = tokenizer(
    f"""<|system|>
You are an n8n workflow generator. Convert natural language descriptions into valid n8n workflow JSON.
<|user|>
{test_prompt}
<|assistant|>
""",
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    top_p=0.95
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("=" * 50)
print("Test Prompt:", test_prompt)
print("=" * 50)
print("Generated Workflow:")
print(generated_text.split("<|assistant|>")[-1].strip())
print("=" * 50)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLaye

Test Prompt: Create a workflow that sends an email notification
Generated Workflow:
public void add more.
 10004.
<|<<<| < 1, 
<|>


 
 2000, and 

 1, and they have a list and 


 10, 20. 
<|assistant<|>

 3, 12
< < < 12.
<|>

-82. 


 
<<| < < < < < < < < < < 


 133, and add an individual, the time, it was a list of the number, and





 
2.



 
<|assion of the list, 0-1,  <
